In [2]:
import requests
from urllib import parse
from lxml import etree
import random
import time
import math
import pandas as pd
import json
from bs4 import BeautifulSoup
import re 

'''
获取拉勾网在线的所有城市名称
页面解析：在源代码可以直接查看到，故为静态网页，可以直接提取，利用BeatuifulSoup解析返回结果
keyword:所要查询关键字，如：'数据分析'
'''
def get_citys(keyword):
    keyword = parse.quote(keyword)
    link = "https://www.lagou.com/jobs/allCity.html?keyword="+keyword+"&px=default&city=%E5%85%A8%E5%9B%BD&positionNum=500+&companyNum=0&isCompanySelected=false&labelWords="
    headers = {
            'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Mobile Safari/537.36'
            }
    r = requests.get(link,headers=headers)
    citys = []
    soup = BeautifulSoup(r.text,'html.parser')
    city_word_list = soup.find_all('ul',class_ ='city_list')
    for i in range(len(city_word_list)):
        city_list = city_word_list[i].find_all('li')
        for j in range(len(city_list)):
            city = city_list[j].a.text.strip()
            citys.append(city)
#             print(city)
    return citys
        

In [3]:
def get_url(city,keyword,gm):
    
    '''
    编码为URL格式,直接传送的地址非URL格式需要利用urllib.parse进行处理
    错误类型：UnicodeEncodeError
     'latin-1' codec can't encode characters in position 87-89: ordinal not in range(256)
    '''
    city = parse.quote(city)
    keyword = parse.quote(keyword)
    gm = parse.quote(gm)
    
    url_parse = "https://www.lagou.com/jobs/positionAjax.json?px=default&city="+city+"&needAddtionalResult=false&gm="+gm
    url_start = "https://www.lagou.com/jobs/list_"+keyword+"?px=default&gm="+ gm + "&city="+city
    return url_parse,url_start

In [4]:
#随机选择执行的IP,防止被封
def get_proxies():
    # IP可以直接在网上找免费的代理直接提取替换
    proxie = [
        "134.249.156.3:82",
        "1.198.72.239:9999",
        "103.26.245.190:43328"]
  
    proxies = {"http":str(random.sample(proxie,1))}
    return proxies
def get_agents():
    # agent中的数据由于用的人已经很多，不一定适合所有人，可以换成自己的User-Agent
    agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
        'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Mobile Safari/537.36'
    ]
    agent = random.sample(agents,1)
    return agent
#定制headers,设置模拟浏览器，反爬
def get_headers(url_start,agent):
    url_start = url_start
    agent = agent
    headers = {
         #所有数据均来自Network-XHR-headers-Requsts Headers
        'Accept': 'application/json, text/javascript, */*; q=0.01',        
        'Referer': url_start,
        #因为本地的User-Agent已经被封锁，此时有了网上的一个虚拟的，为了反爬可以随机使用几个，类似于下面的proxies
#         'Cookie':'user_trace_token=20190515230141-12934d87-ec36-47a4-b6f5-b9fdf6116988; _ga=GA1.2.1677629794.1557932504; _gat=1; LGSID=20190515230142-597afe8c-7722-11e9-99a5-525400f775ce; PRE_UTM=m_cf_cpt_sogou_ztch6; PRE_HOST=; PRE_SITE=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2Flanding-page%2Fpc%2Fposition2.html%3Futm_source%3Dm_cf_cpt_sogou_ztch6; LGUID=20190515230142-597b0038-7722-11e9-99a5-525400f775ce; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557932504; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216abc03944948-07d289ba2ecf5-353166-921600-16abc039450303%22%2C%22%24device_id%22%3A%2216abc03944948-07d289ba2ecf5-353166-921600-16abc039450303%22%7D; sajssdk_2015_cross_new_user=1; _gid=GA1.2.694067267.1557932513; LG_LOGIN_USER_ID=90be76343c3af8869dd6d12563545c28b420c207df6a38cceeb53f0a5d64da34; LG_HAS_LOGIN=1; _putrc=A7C9A275E2B1F6EB123F89F2B170EADC; JSESSIONID=ABAAABAAADEAAFIAD4122C7F738CEEAE9C99959A9756EB3; login=true; unick=%E5%BC%A0%E5%B9%B3; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=1; gate_login_token=450a1f502d12ad1d9f320d846f06d1ad92603529c81fa8ff8921a53bcf54a82d; index_location_city=%E5%8C%97%E4%BA%AC; X_HTTP_TOKEN=c7dc7114524f2f9f15523975516bfb19cb3eab323b; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557932554; LGRID=20190515230231-76ee1774-7722-11e9-99a5-525400f775ce; SEARCH_ID=caaf8eb6597d4ecbbe9a2d807d68e78f',
        'Host':'www.lagou.com',
        'Origin': 'https://www.lagou.com',
        'User-Agent':  str(agent),
        'X-Anit-Forge-Code': '0',
        'X-Anit-Forge-Token': 'None',
        'X-Requested-With': 'XMLHttpRequest'
    }  
    return headers

In [5]:
def get_json(city,keyword,gm,pn):
    #kd是搜索的关键字，若改为“数据分析”即是数据分析相关的数据
    data = { 'first': 'true', 'pn': pn, 'kd': keyword }#keyword若是中文，需要为编码的形式
    url_parse,url_start = get_url(city,keyword,gm)
    agent = get_agents()
    headers = get_headers(url_start,agent)
    proxies = get_proxies()
    print(proxies)
    # 请求首页获取cookies 
    s = requests.Session() 
    s = s.get(url_start, headers=headers, proxies=proxies,timeout=3)   # 请求首页获取cookies 
    cookie = s.cookies  # 为此类别获取的cookies 
    response = requests.post(url_parse,headers=headers,data=data,proxies=proxies,cookies=cookie,timeout=5)
    sleep_time = random.randint(4,5)+random.random()
    time.sleep(sleep_time)
    response.encoding = response.apparent_encoding 
#     print(response.text)
    text = json.loads(response.text)
    return text

In [13]:
'''
获取每个城市的全部数据
'''
def get_city_Pages(city,keyword):
    
    data = { 'first': 'true', 'pn': 1, 'kd': keyword }#keyword若是中文，需要为编码的形式
    city = parse.quote(city)
    keyword = parse.quote(keyword)
    url_parse = "https://www.lagou.com/jobs/positionAjax.json?px=default&city="+city+"&needAddtionalResult=false"
    url_start = "https://www.lagou.com/jobs/list_"+keyword+"?px=default&city="+city
    agent = get_agents()
    headers = get_headers(url_start,agent)
    proxies = get_proxies()
     # 请求首页获取cookies 
    s = requests.Session() 
    s = s.get(url_start, headers=headers, proxies=proxies,timeout=3)   # 请求首页获取cookies 
    cookie = s.cookies  # 为此类别获取的cookies 
    response = requests.post(url_parse,headers=headers,data=data,proxies=proxies,cookies=cookie,timeout=5)
    sleep_time = random.randint(4,5)+random.random()
    time.sleep(sleep_time)
    response.encoding = response.apparent_encoding 
#     print(response.text)
    text = json.loads(response.text)
#     print(text)
    totalcount = 0
    if 'content' in text:
        totalcount = text['content']['positionResult']['totalCount']
    else:
        print("Not exits")
    totalPages = math.ceil(float(totalcount)/15)
    city_un = parse.unquote(city)
    if totalPages > 0 :
        print("%s共%s条,可分为%s页"%(city_un,totalcount,totalPages))
#         sleep_time = random.randint(5,15)+random.random()
#         time.sleep(sleep_time)#keyerror问题的解决
#         print("停留%s"%sleep_time)
    else:
#         city_un = parse.quote(city)
        print("%s-----无数据"%(city_un))
#     print("共%s条,可分为%s页"%(totalcount,totalPages))
    return totalPages

In [7]:
def get_totalPages(city,keyword,gm):
    city = city
    keyword = keyword
    totalcount = 0
#     pn = 1
    text = get_json(city,keyword,gm,'1')
    if 'content' in text:
        totalcount = text['content']['positionResult']['totalCount']
    else:
        print("Not exits")
    totalPages = math.ceil(float(totalcount)/15)
    if totalPages > 0 :
        print("%s公司规模为%s共%s条,可分为%s页"%(city,gm,totalcount,totalPages))        
#         sleep_time = random.randint(5,15)+random.random()
#         time.sleep(sleep_time)#keyerror问题的解决
#         print("停留%s"%sleep_time)
    else:
        print("%s公司规模为%s---无数据"%(city,gm))
      
    return totalPages

In [8]:
def get_info(city,keyword,gm,totalPages):
    city = city
    keyword = keyword
    totalPages = totalPages
    position_info_all = []#存储数据
    for page in range(1,totalPages+1):
        time_start = time.time()
        text = get_json(city,keyword,gm,page)
#         print(text)
#         print(proxies)
        if 'content'in text:
            info = text["content"]["positionResult"]["result"] 
            for i in info:
                position_info_single = []
                position_info_single.append(i['positionId'])#职位编号
                position_info_single.append(i['positionName'])#职位名称
                position_info_single.append(i['salary'])#薪酬
                position_info_single.append(i['workYear'])#工作经验
                position_info_single.append(i['skillLables'])#技能要求
                position_info_single.append(i['positionAdvantage'])#职位优势
                position_info_single.append(i['education'])#学历要求
                position_info_single.append(i['jobNature'])#工作性质
                position_info_single.append(i['createTime'])#发布时间
                position_info_single.append(i['companyFullName'])#公司
                position_info_single.append(i['city'])#城市
                position_info_single.append(i['companySize'])#公司规模
                position_info_single.append(i['district'])#区域
                position_info_single.append(i['financeStage'])#融资情况
                position_info_single.append(i['firstType'])#公司类别
                position_info_single.append(i['industryField'])#涉及领域
                position_info_single.append(i['isSchoolJob'])#是否校招
                position_info_single.append(i['subwayline'])#地铁
                position_info_single.append(i['stationname'])#站点
                position_info_single.append(i['latitude'])#经度
                position_info_single.append(i['longitude'])#纬度
                position_info_all.append(position_info_single) 
            print("第%s页爬取成功,position_info_all现有数据%s行"%(str(page),str(len(position_info_all))))
        else:
            print("Not exits")
        sleep_time = random.randint(5,15)+random.random()
        time.sleep(sleep_time)#keyerror问题的解决
        time_end = time.time()
        on_time = time_end-time_start
        print("本页爬行时间%s"%str(on_time))
    return position_info_all

In [ ]:
def main():
    #公司规模
    gms = [
        '2000人以上',
        '少于15人',
        '15-50人',
        '50-150人',
        '150-500人',
        '500-2000人' 
    ]    
    keyword = '数据分析'
    citys = get_citys(keyword)
#     citys = ['北京']
    position_info = []
    for i in range(len(citys)):
        city = citys[i]
        cityPages = get_city_Pages(city,keyword)
        if cityPages > 0 :
            for j in range(len(gms)):
                gm = gms[j]
                totalPages = get_totalPages(city,keyword,gm)
                if totalPages>0:
                    position_info_all = get_info(city,keyword,gm,totalPages)
                    position_info.append(position_info_all)
    #                 print(position_info_all)
                    print("%s的公司规模%s的数据爬完"%(city,gm))
                else:
    #                 print("%s的公司规模%s无数据"%(city,gm))
                    sleep_time = random.randint(5,15)+random.random()
                    time.sleep(sleep_time)#keyerror问题的解决
                    print("停留%s"%sleep_time)
            print("position_info有%s条数据"%len(position_info))
            print("%s的全部数据爬取成功"%city)
            sleep_time = random.randint(5,15)+random.random()
            time.sleep(sleep_time)#keyerror问题的解决
            print("停留%s"%sleep_time)
        else:
            sleep_time = random.randint(5,15)+random.random()
            time.sleep(sleep_time)#keyerror问题的解决
            print("停留%s"%sleep_time)
    columns = ['职位编号','职位名称','薪酬','工作经验','技能要求','职位优势','学历要求','工作形式','发布时间',
               '公司','城市','规模','区域','融资情况','公司类别','涉及领域','是否校招',
              '地铁','站点','经度','纬度']
    df = pd.DataFrame(data = position_info_all,columns =columns )
    df.to_csv('lagou_data_page.csv',index = False,encoding="utf_8_sig")
    print("全部数据存储成功（CSV格式）！") 
    print(position_info_all)

if __name__ == '__main__':
    main()

安庆共1条,可分为1页
{'http': "['115.28.148.192:8118']"}
安庆公司规模为2000人以上---无数据
停留7.621036872046908
{'http': "['218.64.69.79:8080']"}
安庆公司规模为少于15人---无数据
停留12.682957568394892
{'http': "['163.125.68.135:8888 ']"}
安庆公司规模为15-50人---无数据
停留14.01956860660513
{'http': "['218.64.69.79:8080']"}
安庆公司规模为50-150人---无数据
停留5.747118274892129
{'http': "['1.199.30.167:9999 ']"}
安庆公司规模为150-500人共1条,可分为1页
{'http': "['120.83.108.77:9999 ']"}
第1页爬取成功,position_info_all现有数据1行
本页爬行时间14.79757022857666
安庆的公司规模150-500人的数据爬完
{'http': "['112.85.164.68:9999']"}
安庆公司规模为500-2000人---无数据
停留15.455584764709632
position_info有1
安庆的全部数据爬取成功
停留10.107098117786421
澳门特别行政区-----无数据
停留8.157623424753595
安康-----无数据
停留11.453615679459165
鞍山-----无数据
停留12.428510394537883
安阳-----无数据
停留15.194597831851825
安顺-----无数据
停留6.493755356505584
北京共5423条,可分为362页
{'http': "['1.192.243.209:9999']"}
北京公司规模为2000人以上共345条,可分为23页
{'http': "['116.208.54.33:9999 ']"}
第1页爬取成功,position_info_all现有数据15行
本页爬行时间19.5901939868927
{'http': "['183.129.207.86:11206 ']"}
第2页爬取成功,posi

成都公司规模为500-2000人共7条,可分为1页
{'http': "['222.89.32.187:9999 ']"}
第1页爬取成功,position_info_all现有数据7行
本页爬行时间18.739002466201782
成都的公司规模500-2000人的数据爬完
position_info有14
成都的全部数据爬取成功
停留9.3236670790171
长沙共16条,可分为2页
{'http': "['112.87.68.236:9999 ']"}
长沙公司规模为2000人以上共5条,可分为1页
{'http': "['1.192.243.209:9999']"}
第1页爬取成功,position_info_all现有数据5行
本页爬行时间12.8500337600708
长沙的公司规模2000人以上的数据爬完
{'http': "['171.80.2.110:9999']"}
长沙公司规模为少于15人---无数据
停留12.66636741586509
{'http': "['120.83.108.77:9999 ']"}
长沙公司规模为15-50人共3条,可分为1页
{'http': "['218.64.69.79:8080']"}
第1页爬取成功,position_info_all现有数据3行
本页爬行时间12.777708530426025
长沙的公司规模15-50人的数据爬完
{'http': "['1.198.72.193:9999']"}
Not exits
长沙公司规模为50-150人---无数据
停留9.162774503290803
{'http': "['171.80.2.110:9999']"}
长沙公司规模为150-500人共6条,可分为1页
{'http': "['171.80.2.110:9999']"}
第1页爬取成功,position_info_all现有数据6行
本页爬行时间18.694823026657104
长沙的公司规模150-500人的数据爬完
{'http': "['1.198.72.193:9999']"}
长沙公司规模为500-2000人共2条,可分为1页
{'http': "['116.208.54.212:9999 ']"}
第1页爬取成功,position_info_all现有数据2行
本页

本页爬行时间19.153416633605957
广州的公司规模少于15人的数据爬完
{'http': "['14.115.104.97:808 ']"}
广州公司规模为15-50人共13条,可分为1页
{'http': "['1.198.72.193:9999']"}
第1页爬取成功,position_info_all现有数据13行
本页爬行时间19.83248233795166
广州的公司规模15-50人的数据爬完
{'http': "['1.192.243.166:9999 ']"}
广州公司规模为50-150人共26条,可分为2页
{'http': "['218.64.69.79:8080']"}
第1页爬取成功,position_info_all现有数据15行
本页爬行时间19.708940029144287
{'http': "['116.208.54.33:9999 ']"}
第2页爬取成功,position_info_all现有数据26行
本页爬行时间19.019227027893066
广州的公司规模50-150人的数据爬完
{'http': "['112.85.164.68:9999']"}
广州公司规模为150-500人共53条,可分为4页
{'http': "['183.129.207.86:11206 ']"}
第1页爬取成功,position_info_all现有数据15行
本页爬行时间17.35615563392639
{'http': "['1.198.72.193:9999']"}
第2页爬取成功,position_info_all现有数据30行
本页爬行时间20.661059141159058
{'http': "['1.198.72.193:9999']"}
第3页爬取成功,position_info_all现有数据45行
本页爬行时间16.995171785354614
{'http': "['183.129.207.86:11206 ']"}
第4页爬取成功,position_info_all现有数据53行
本页爬行时间21.400070667266846
广州的公司规模150-500人的数据爬完
{'http': "['1.198.72.239:9999']"}
广州公司规模为500-2000人共48条,可分为4页
{'h